# Cartesian

Welcome to the tutorial for ChemCoord (http://chemcoord.readthedocs.org/).

The manipulation of the coordinates is a lot easier, if you can view them on the fly.
So please install a molecule viewer, which opens xyz-files. 
A non complete list includes:
[molcas gv](http://www.molcas.org/GV/),
[avogadro](https://avogadro.cc/),
[vmd](http://www.ks.uiuc.edu/Research/vmd/), and
[pymol](https://www.pymol.org/)

In [1]:
import chemcoord as cc
import pandas as pd
import numpy as np
import time

In [2]:
water = cc.Cartesian.read_xyz('water_dimer.xyz', start_index=1)

Let's have a look at it:

In [3]:
water

,atom,x,y,z
1,O,0.000000,0.0,0.000000
2,H,0.758602,0.0,0.504284
3,H,0.260455,0.0,-0.872893
4,O,3.000000,0.5,0.000000
5,H,3.758602,0.5,0.504284
6,H,3.260455,0.5,-0.872893


It is also possible to open it with an external viewer. I use Molcas ``gv.exe`` so you have to change it accordingly to your program of choice.

In [4]:
water.view(viewer='gv.exe')

To make this setting permament, execute:

In [5]:
cc.settings['defaults']['viewer'] = 'gv.exe'  # replace by your viewer of choice

# Slicing

The slicing operations are the same as for ``pandas.DataFrames``. (http://pandas.pydata.org/pandas-docs/stable/indexing.html)

If the ``'x'`` axis is of particular interest you can slice it out with:

In [6]:
water['x']
# or explicit label based indexing
water.loc[:, 'x']
# or explicit integer based indexing
water.iloc[:, 1]

1    0.000000
2    0.758602
3    0.260455
4    3.000000
5    3.758602
6    3.260455
Name: x, dtype: float64

With boolean slicing it is very easy to  cut all the oxygens away:

In [7]:
water[water['atom'] != 'O'].view()

This can be combined with other selections:

In [8]:
water[(water['atom'] != 'O') & (water['x'] < 1)].view()

### Returned type

The indexing behaves like Indexing and Selecting data in
[Pandas](<http://pandas.pydata.org/pandas-docs/stable/indexing.html>).
You can slice with `Cartesian.loc[key]`, `Cartesian.iloc[keys]`, and `Cartesian[key]`.
The only question is about the return type.
If the information in the columns is enough to draw a molecule,
an instance of the own class (e.g. `Cartesian`)
is returned.
If the information in the columns is not enough to draw a molecule, there 
are two cases to consider:

* A `pandas.Series` instance is returned for one dimensional slices
* A `pandas.DataFrame` instance is returned in all other cases:

        molecule.loc[:, ['atom', 'x', 'y', 'z']] returns a `Cartesian`.

        molecule.loc[:, ['atom', 'x']]`` returns a `pandas.DataFrame`.

        molecule.loc[:, 'atom']`` returns a `pandas.Series`.

# Chemical bonds

One really important method is ``get_bonds()``. 
It returns a connectivity table, which is represented by a dictionary.
Each index points to set of indices, that are connected to it.

In [9]:
water.get_bonds()

{1: {2, 3}, 2: {1}, 3: {1}, 4: {5, 6}, 5: {4}, 6: {4}}

In order to play around with the coordinationsphere, we have to import another molecule

In [ ]:
small = cc.Cartesian.read_xyz('MIL53_small.xyz', start_index=1)
middle = cc.Cartesian.read_xyz('MIL53_middle.xyz', start_index=1)

Let's explore the coordinationsphere of the Cr atom with the index 7.

In [ ]:
for i in range(3):
    small.connected_to(7, n_sphere=i).view()
    time.sleep(1)

We can also partition the molecule into subsets of the same chemical environment

In [12]:
small.partition_chem_env()

{('C', frozenset({('C', 4), ('Cr', 2), ('H', 7), ('O', 7)})): {2, 5, 14, 21},
 ('C', frozenset({('C', 6), ('Cr', 2), ('H', 8), ('O', 11)})): {10, 17},
 ('C', frozenset({('C', 1), ('Cr', 2), ('H', 3), ('O', 11)})): {24, 32},
 ('C', frozenset({('C', 1), ('Cr', 1), ('H', 4), ('O', 7)})): {27, 31, 33, 34},
 ('Cr', frozenset({('C', 10), ('Cr', 1), ('H', 19), ('O', 13)})): {6, 11},
 ('H', frozenset({('C', 2), ('Cr', 2), ('H', 2), ('O', 2)})): {25,
  26,
  35,
  36,
  37,
  38},
 ('H', frozenset({('C', 2), ('Cr', 1), ('H', 3), ('O', 2)})): {28,
  29,
  30,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  49,
  50,
  51,
  52},
 ('H', frozenset({('C', 4), ('Cr', 2), ('H', 2), ('O', 6)})): {48, 54, 55, 56},
 ('H', frozenset({('C', 6), ('Cr', 2), ('H', 4), ('O', 11)})): {53},
 ('O', frozenset({('C', 2), ('Cr', 1), ('H', 4), ('O', 6)})): {1, 4, 22, 23},
 ('O', frozenset({('C', 8), ('Cr', 2), ('H', 3), ('O', 12)})): {3, 18},
 ('O', frozenset({('C', 12), ('Cr', 2), ('H', 5), ('O', 14)})): {7

Another task we can easily solve is: making cuts of different geometrical shape, while preserving covalent bonds.

In [13]:
middle.view()
middle.cutsphere(radius=5, preserve_bonds=False).view()

# Binary operators

### Mathematical Operations:

Binary operators are supported in the logic of the scipy stack, but you need
python3.x for using the matrix multiplication operator ``@``.

The general rule is that mathematical operations using the binary operators
``+ - * / @`` and the unary operatos ``+ - abs``
are only applied to the ``['x', 'y', 'z']`` columns.

**Addition/Subtraction/Multiplication/Division**:
If you add a scalar to a Cartesian it is added elementwise onto the
``['x', 'y', 'z']`` columns.
If you add a 3-dimensional vector, list, tuple... the first element of this
vector is added elementwise to the ``'x'`` column of the
Cartesian instance and so on.
The last possibility is to add a matrix with
``shape=(len(Cartesian), 3)`` which is again added elementwise.
The same rules are true for subtraction, division and multiplication.

**Matrixmultiplication**:
Only leftsided multiplication with a matrix of ``shape=(n, 3)``,
where ``n`` is a natural number, is supported.
The usual usecase is for example
``np.diag([1, 1, -1]) @ cartesian_instance``
to mirror on the x-y plane.

In [14]:
(water + 3).view()

In [16]:
rot_mat = cc.utilities.algebra_utilities.rotation_matrix
(rot_mat([1, 0, 0], np.pi / 2) @ water).view()
# If you use python2.x the @ operator is not supported then you have to use:
# cc.xyz_functions.dot(rot_mat([1, 0, 0], np.pi / 2), water).view()

### Comparison:

The comparison operators ``==`` and ``!=`` are supported and require molecules indexed in the same way:

In some cases it is better to test for numerical equality $ |a - b| < \epsilon$. This is done using
``allclose`` or ``isclose`` (elementwise)

In [17]:
water == water + 1e-15

,atom,x,y,z
1,True,False,False,False
2,True,False,False,False
3,True,False,False,False
4,True,False,False,False
5,True,False,False,False
6,True,False,False,False


In [18]:
cc.xyz_functions.isclose(water, water + 1e-15)

,atom,x,y,z
1,True,True,True,True
2,True,True,True,True
3,True,True,True,True
4,True,True,True,True
5,True,True,True,True
6,True,True,True,True


In [19]:
cc.xyz_functions.allclose(water, water + 1e-15)

True

# Symbolic evaluation

# Symmetry